In [25]:
# This Script connects to the Meraki API and retrieves the L3 and L7 Firwallrules and puts it in an excel sheet.

# Install the relevant modules. If you are using VS Code you can run these commands, without the preceding %, via a terminal. NB: Run `pip install meraki==` to find the latest version of the Meraki SDK.
%pip install meraki
%pip install python-dotenv
%pip install openpyxl
%pip install pandas

import pandas as pd
import meraki
import os
import json
import datetime

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [26]:
# Get the current date and time
now = datetime.datetime.now()

# Format the date and time as a string
now_str = now.strftime("%Y%m%d-%H%M")

In [27]:
# Loading environment variables

from dotenv import load_dotenv
# Load the .env from the current folder
load_dotenv()

# Load your Meraki Dashboard API key from environment variables
API_KEY = os.getenv("MERAKI_DASHBOARD_API_KEY")
# print(API_KEY)

# Initialize the Dashboard connection
dashboard = meraki.DashboardAPI(API_KEY)

# Specify the organization ID
organization_id = os.getenv("MERAKI_ORGANIZATION_ID")

# Specify the network ID
network_id = os.getenv("MERAKI_NETWORK_ID")

2023-11-13 17:46:48       meraki:     INFO > Meraki dashboard API session initialized with these parameters: {'version': '1.39.0', 'api_key': '************************************f4ea', 'base_url': 'https://api.meraki.com/api/v1', 'single_request_timeout': 60, 'certificate_path': '', 'requests_proxy': '', 'wait_on_rate_limit': True, 'nginx_429_retry_wait_time': 60, 'action_batch_retry_wait_time': 60, 'network_delete_retry_wait_time': 240, 'retry_4xx_error': False, 'retry_4xx_error_wait_time': 60, 'maximum_retries': 2, 'simulate': False, 'be_geo_id': None, 'caller': None, 'use_iterator_for_get_pages': False}


In [28]:
# Write an excel file with all L3 fw rules
# Create the filename for l3 firewall rules
meraki_fw_l3 = f"{now_str}_meraki-fw-l3-rules.xlsx"

# Get the L3 firewall rules
l3_firewall_rules = dashboard.appliance.getNetworkApplianceFirewallL3FirewallRules(network_id)

# Convert the L3 firewall rules to a pandas DataFrame
l3_firewall_rules_df = pd.DataFrame(l3_firewall_rules['rules'])

# Index zurücksetzen, um eine neue Spalte zu erstellen
l3_firewall_rules_df.reset_index(inplace=True)

# Index um 1 erhöhen, um bei 1 zu beginnen
l3_firewall_rules_df['index'] = l3_firewall_rules_df['index'] + 1

# Umbenennen der 'index'-Spalte in 'Rule Number'
l3_firewall_rules_df.rename(columns={'index': 'Rule Number'}, inplace=True)

# Write the DataFrame to an Excel file
l3_firewall_rules_df.to_excel(meraki_fw_l3, index=False)

2023-11-13 17:46:48       meraki:     INFO > GET https://api.meraki.com/api/v1/networks/L_671599294431637280/appliance/firewall/l3FirewallRules
2023-11-13 17:46:49       meraki:     INFO > appliance, getNetworkApplianceFirewallL3FirewallRules - 200 OK


In [29]:
# Group Policy Objects
# Create the filename for Group Policies
meraki_opo = f"{now_str}_meraki-network-group-policies.xlsx"

# Get the Group Policy objects
group_policies = dashboard.networks.getNetworkGroupPolicies(network_id)

# Convert the Group Policy objects to a pandas DataFrame
group_policies_df = pd.DataFrame(group_policies)

# Write the DataFrame to an Excel file
group_policies_df.to_excel(meraki_opo, index=False)
# Organization Policy Objects
# Create the filename for Organization Policy Objects
meraki_opo = f"{now_str}_meraki-organization-policy-objects.xlsx"

# Get the Organization Policy Objects
# Achtung: Methode 'getOrganizationPolicyObjects' ist ein Platzhalter
# und sollte durch die korrekte API-Methode ersetzt werden.
policy_objects = dashboard.organizations.getOrganizationPolicyObjects(organization_id)

# Convert the Organization Policy Objects to a pandas DataFrame
policy_objects_df = pd.DataFrame(policy_objects)

# Write the DataFrame to an Excel file
policy_objects_df.to_excel(meraki_opo, index=False)

#-------

def resolve_grp_ids(grp_id, policy_objects_df):

    # Debugging: Ausgabe der grp_id
    print(f"Gruppen-ID: {grp_id}")

    # Überprüfe, ob die Gruppen-ID in der Liste der Gruppen-IDs für jedes Objekt vorhanden ist
    matching_objs = policy_objects_df[policy_objects_df['groupIds'].apply(lambda x: grp_id in x)]

    # Debugging: Ausgabe der gefundenen Objekte
    print(f"Gefundene Objekte für Gruppen-ID {grp_id}:")
    print(matching_objs)

    # Extrahiere die Details für jedes gefundene Objekt
    details = matching_objs[['name', 'cidr']]

    # Konvertiere die Details in das gewünschte Format
    resolved_details = details.apply(lambda x: f"{x['name']} ({x['cidr']})", axis=1).tolist()

    # Debugging: Ausgabe der aufgelösten Details
    print(f"Aufgelöste Details für Gruppen-ID {grp_id}:")
    print(resolved_details)

    return resolved_details


    # Finde alle OBJs, die zu dieser GRP gehören
    obj_ids = policy_objects_df[policy_objects_df['groupIds'] == grp_id]['id']
    # Für jedes OBJ, sammle den Namen, die Bezeichnung und CIDR-Informationen
    details = policy_objects_df[policy_objects_df['id'].isin(obj_ids)][['name', 'cidr']]
    return details.apply(lambda x: f"{x['name']} ({x['cidr']})", axis=1).tolist()

def resolve_obj_grp(firewall_df, policy_objects_df):
    # Hilfsfunktion zur Auflösung der IDs
    def resolve_ids(ids, policy_objects_df):
        names = []
        for id in ids:
            if 'GRP' in id:
                # Extrahiere und löse GRP ID auf
                grp_id = id.replace('GRP(', '').replace(')', '')
                grp_details = resolve_grp_ids(grp_id, policy_objects_df)
                names.extend(grp_details)
            elif 'OBJ' in id:
                # Extrahiere und löse OBJ ID auf
                obj_id = id.replace('OBJ(', '').replace(')', '')
                obj = policy_objects_df[policy_objects_df['id'] == obj_id]
                if not obj.empty:
                    names.append(f"{obj.iloc[0]['name']} ({obj.iloc[0]['cidr']})")
            elif 'VLAN' in id:
                # Behandle VLAN Einträge
                names.append(id)
            elif id == 'Any':
                # Behandle "Any"
                names.append('Any')
            else:
                # Behandle als direkte IP-Adresse oder CIDR-Bereich
                names.append(id)
        return ', '.join(names)

    # Spalten für OBJ, GRP, VLAN, IP und Any vorbereiten
    firewall_df['srcCidrIds'] = firewall_df['srcCidr'].str.findall(r'(OBJ\(\d+\)|GRP\(\d+\)|VLAN\(\d+\).*|Any|\d{1,3}(?:\.\d{1,3}){3}(?:/\d{1,2})?)')
    firewall_df['destCidrIds'] = firewall_df['destCidr'].str.findall(r'(OBJ\(\d+\)|GRP\(\d+\)|VLAN\(\d+\).*|Any|\d{1,3}(?:\.\d{1,3}){3}(?:/\d{1,2})?)')

    # Auflösen der Namen und CIDRs
    firewall_df['resolvedSrcCidr'] = firewall_df['srcCidrIds'].apply(
        lambda ids: resolve_ids(ids, policy_objects_df)
    )
    firewall_df['resolvedDestCidr'] = firewall_df['destCidrIds'].apply(
        lambda ids: resolve_ids(ids, policy_objects_df)
    )

    return firewall_df

# Funktion aufrufen und DataFrame aktualisieren
l3_firewall_rules_df = resolve_obj_grp(l3_firewall_rules_df, policy_objects_df)

# Spalten in der gewünschten Reihenfolge neu anordnen und Excel-Datei generieren
l3_firewall_rules_df = l3_firewall_rules_df[['Rule Number', 'comment', 'policy', 'protocol', 'srcPort', 'resolvedSrcCidr', 'srcCidrIds', 'destPort', 'resolvedDestCidr', 'destCidr', 'syslogEnabled']]
# DataFrame in eine neue Excel-Datei schreiben
l3_firewall_rules_df.to_excel(f"{now_str}_resolved_l3_firewall_rules.xlsx", index=False)


2023-11-13 17:46:49       meraki:     INFO > GET https://api.meraki.com/api/v1/networks/L_671599294431637280/groupPolicies


2023-11-13 17:46:50       meraki:     INFO > networks, getNetworkGroupPolicies - 200 OK
2023-11-13 17:46:50       meraki:     INFO > GET https://api.meraki.com/api/v1/organizations/563611/policyObjects
2023-11-13 17:46:50       meraki:     INFO > organizations, getOrganizationPolicyObjects; page 1 - 200 OK


Gruppen-ID: 743656888469553646
Gefundene Objekte für Gruppen-ID 743656888469553646:
                   id            name category  type             cidr  \
7  743656888469555291    vlan-dhcp-20  network  cidr  192.168.20.0/24   
8  743656888469555292  vlan-default-1  network  cidr   192.168.0.0/24   
9  743656888469555293    vlan-roth-30  network  cidr  192.168.30.0/24   

              createdAt             updatedAt  \
7  2022-12-16T10:52:03Z  2022-12-16T10:52:03Z   
8  2022-12-16T10:52:38Z  2022-12-16T11:09:02Z   
9  2022-12-16T10:53:04Z  2022-12-16T10:53:04Z   

                                   groupIds              networkIds  
7                      [743656888469553646]                      []  
8  [743656888469553646, 743656888469553647]  [L_671599294431637280]  
9                      [743656888469553646]                      []  
Aufgelöste Details für Gruppen-ID 743656888469553646:
['vlan-dhcp-20 (192.168.20.0/24)', 'vlan-default-1 (192.168.0.0/24)', 'vlan-roth-30 (192.168

In [30]:
print(policy_objects_df.columns)
print(group_policies_df.columns)

Index(['id', 'name', 'category', 'type', 'cidr', 'createdAt', 'updatedAt',
       'groupIds', 'networkIds'],
      dtype='object')
Index(['groupPolicyId', 'name', 'scheduling', 'bandwidth',
       'firewallAndTrafficShaping', 'contentFiltering', 'splashAuthSettings',
       'vlanTagging', 'bonjourForwarding'],
      dtype='object')


In [31]:
print(l3_firewall_rules_df.columns)

Index(['Rule Number', 'comment', 'policy', 'protocol', 'srcPort',
       'resolvedSrcCidr', 'srcCidrIds', 'destPort', 'resolvedDestCidr',
       'destCidr', 'syslogEnabled'],
      dtype='object')
